<a href="https://colab.research.google.com/github/vindinudecosta/mocktail_machine_server_code/blob/main/Natural_language_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import json

import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('intents.json', 'r') as f:
    data = json.load(f)

print(data.keys())
print(type(data['intents']))
print(len(data['intents']))
print(data['intents'][0].keys())
data['intents'][-1]

dict_keys(['intents'])
<class 'list'>
285
dict_keys(['intent', 'text', 'responses', 'extension', 'context', 'entityType', 'entities'])


{'intent': 'OrderRejection',
 'text': ['No'],
 'responses': ['order cancelled'],
 'extension': {'function': '', 'entities': False, 'responses': []},
 'context': {'in': '', 'out': '', 'clear': False},
 'entityType': 'NA',
 'entities': []}

In [ ]:
def clean(line):
    cleaned_line = ''
    for char in line:
        if char.isalpha():
            cleaned_line += char
        else:
            cleaned_line += ' '
    cleaned_line = ' '.join(cleaned_line.split())
    return cleaned_line

In [ ]:
#list of intents
intents = []
unique_intents = []
#all text data to create a corpus
text_input= []
#dictionary mapping intent with appropriate response
response_for_intent = {}
for intent in data['intents']:
    #list of unique intents
    if intent['intent'] not in unique_intents:
        unique_intents.append(intent['intent'])
    for text in intent['text']:
        #cleaning is done before adding text to corpus
        text_input.append(clean(text))
        intents.append(intent['intent'])
    if intent['intent'] not in response_for_intent:
        response_for_intent[intent['intent']] = []
    for response in intent['responses']:
        response_for_intent[intent['intent']].append(response)

In [ ]:
print("Intent :",intents[0])
print("Number of Intent:",len(intents))
print("Sample Input:", text_input[0])
print('Length of text_input:',len(text_input))
print("Sample Response: ", response_for_intent[intents[0]])

Intent : DrinkSelection1
Number of Intent: 285
Sample Input: Make drink one
Length of text_input: 285
Sample Response:  ['Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Let's mix it up!", 'Virgin Mojito coming right up! Stay fresh.', 'Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Let's mix it up!", 'Virgin Mojito coming right up! Stay fresh.', 'Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Let's mix it up!", 'Virgin Mojito coming right up! Stay fresh.', 'Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Let's mix it up!", 'Virgin Mojito coming right up! Stay fresh.', 'Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Let's mix it up!", 'Virgin Mojito coming right up! Stay fresh.', 'Virgin Mojito selected. Preparing your refreshing drink!', "You've chosen a Virgin Mojito. Le

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

In [ ]:
tokenizer = Tokenizer(filters='',oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')
print('Shape of Input Sequence:',padded_sequences.shape)
padded_sequences[:5]

Shape of Input Sequence: (285, 5)


array([[ 0,  0,  3,  2, 12],
       [ 0, 10,  2,  6, 12],
       [ 0,  4,  5,  2, 12],
       [ 7,  8,  3,  2, 12],
       [ 0,  0,  2, 12,  9]], dtype=int32)

In [ ]:
intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
print('Number of Intents :',num_classes)

# Convert intent_to_index to index_to_intent
index_to_intent = {index: intent for intent, index in intent_to_index.items()}
index_to_intent

Number of Intents : 25


{0: 'DrinkSelection1',
 1: 'DrinkSelection2',
 2: 'DrinkSelection3',
 3: 'DrinkSelection4',
 4: 'DrinkSelection5',
 5: 'DrinkSelection6',
 6: 'DrinkSelection7',
 7: 'DrinkSelection8',
 8: 'DrinkSelection9',
 9: 'DrinkSelection10',
 10: 'DrinkSelection11',
 11: 'DrinkSelection12',
 12: 'DrinkSelection13',
 13: 'DrinkSelection14',
 14: 'DrinkSelection15',
 15: 'DrinkSelection16',
 16: 'DrinkSelection17',
 17: 'DrinkSelection18',
 18: 'DrinkSelection19',
 19: 'DrinkSelection20',
 20: 'High',
 21: 'medium',
 22: 'Low',
 23: 'OrderConfirm',
 24: 'OrderRejection'}

In [ ]:
categorical_vec = tf.keras.utils.to_categorical(categorical_target,
                                                num_classes=num_classes)

categorical_vec = categorical_vec.astype('int32')

print('Shape of Ca',categorical_vec.shape)
categorical_vec[:5]

Shape of Ca (285, 25)


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]], dtype=int32)

In [ ]:
epochs=150
embed_dim=300
lstm_num=50
output_dim=categorical_vec.shape[1]
input_dim=len(unique_intents)
print("Input Dimension :{},\nOutput Dimension :{}".format(input_dim,output_dim))

Input Dimension :25,
Output Dimension :25


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1,
                              output_dim=embed_dim,
                              input_length=padded_sequences.shape[1]),  # Input shape specified here
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_num, dropout=0.1)),
    tf.keras.layers.Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1,
                              output_dim=embed_dim,
                              input_length=padded_sequences.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_num, return_sequences=True, dropout=0.2)),
    tf.keras.layers.LSTM(lstm_num, dropout=0.2),
    tf.keras.layers.Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=0)

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 5, 300)              │          10,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 100)                 │         140,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           1,275 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 472,577 (1.80 MB)

 Trainable params: 157,525 (615.33 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 315,052 (1.20 MB)

In [ ]:
test_text_inputs = ["one",
                    "three",
                    "four",
                    "five",
                    "six"]

test_intents = ["DrinkSelection1",
                "DrinkSelection3",
                "DrinkSelection4",
                "DrinkSelection5",
                "DrinkSelection6"
             ]

test_sequences = tokenizer.texts_to_sequences(test_text_inputs)
test_padded_sequences = pad_sequences(test_sequences,  padding='pre')
test_labels = np.array([unique_intents.index(intent) for intent in test_intents])
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)
loss, accuracy = model.evaluate(test_padded_sequences, test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step - accuracy: 1.0000 - loss: 0.0397


In [ ]:
def response(sentence):
    sent_tokens = []
    # Split the input sentence into words
    words = sentence.split()
    # Convert words to their corresponding word indices
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            # Handle unknown words
            sent_tokens.append(tokenizer.word_index['<unk>'])
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    #predict numerical category
    pred = model(sent_tokens)
    #category to intent
    pred_class = np.argmax(pred.numpy(), axis=1)
    # random response to that intent
    return random.choice(
        response_for_intent[index_to_intent[pred_class[0]]]), index_to_intent[pred_class[0]]

In [ ]:
import re

def convert_digits_to_words(text):
    # Dictionary of numbers to words (1 to 20)
    number_words = {
        '1': 'one',
        '2': 'two',
        '3': 'three',
        '4': 'four',
        '5': 'five',
        '6': 'six',
        '7': 'seven',
        '8': 'eight',
        '9': 'nine',
        '10': 'ten',
        '11': 'eleven',
        '12': 'twelve',
        '13': 'thirteen',
        '14': 'fourteen',
        '15': 'fifteen',
        '16': 'sixteen',
        '17': 'seventeen',
        '18': 'eighteen',
        '19': 'nineteen',
        '20': 'twenty'
    }

    # Replace digits (from 1 to 20) with words
    text = re.sub(r'\b(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20)\b',
                  lambda match: number_words[match.group(0)], text)

    return text

In [ ]:
print("Note: Enter 'quit' to break the loop.")



while True:
    query = input('You: ')

    if query.lower() == 'quit':
        break

    # Apply digit conversion preprocessing before passing the query to the model
    query = convert_digits_to_words(query)

    bot_response, typ = response(query)  # Assuming response is your function for bot reply
    print('Geek: {} -- TYPE: {}'.format(bot_response, typ))
    print()

Note: Enter 'quit' to break the loop.
You: i want drink 1
Geek: Virgin Mojito coming right up! Stay fresh. -- TYPE: DrinkSelection1



KeyboardInterrupt: Interrupted by user

In [ ]:
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print('Geek: {} -- TYPE: {}'.format(bot_response, typ))
    print()

Note: Enter 'quit' to break the loop.
You: 1
Geek: Nojito selected. A mojito without the buzz! -- TYPE: DrinkSelection5

You: i need drink number 1
Geek: You've chosen Mango Mule. A delicious twist is coming. -- TYPE: DrinkSelection7

You: i need drink number one
Geek: You've chosen a Virgin Mojito. Let's mix it up! -- TYPE: DrinkSelection1

You: i need drink number eleven
Geek: You've chosen Arnold Palmer. The perfect balance of flavors. -- TYPE: DrinkSelection11

You: i need drink number 11
Geek: Watermelon Lemonade selected. A fruity twist on a classic! -- TYPE: DrinkSelection14



KeyboardInterrupt: Interrupted by user

In [ ]:

model.save('intent_recognition_model_V1.h5')